In [ ]:

# 꽉차있는 폭탄을 피하면서, 전체 타일을 뒤집는 게임입니다. 숫자는 그 주위에 차 있는 폭탄 수를 나타냅니다.


import sys
from math import floor
import pygame
from random import randint
from pygame.locals import QUIT,MOUSEBUTTONDOWN

WIDTH=20
HEIGHT= 15
SIZE = 50
EMPTY= 0
NUM_OF_BOMBS=20
BOMB=1
OPEN_COUNT= 0
CHECKED =[ [0 for _ in range(WIDTH)] for _ in range(HEIGHT)]
OPENED = 2

pygame.init()
SURFACE = pygame.display.set_mode((WIDTH*SIZE,HEIGHT*SIZE))
FPSCLOCK = pygame.time.Clock()

def num_of_bomb(field,x_pos,y_pos): # xpos와 ypos는 마우스클릭 위치가 아니라.  field 리스트에서 위치를 말하는거임
    # 주위에 있는 폭탄의 수를 반환한다.
    count = 0
    for yoffset in range(-1,2):
        for xoffset in range(-1,2):
            xpos,ypos= (x_pos+xoffset, y_pos+yoffset)
            if 0 <= xpos < WIDTH  and 0 <= ypos < HEIGHT and field[ypos][xpos] == BOMB:
                count += 1
    return count

def open_tile(field,x_pos,y_pos):   # xpos와 ypos는 마우스클릭 위치가 아니라.  field 리스트에서 위치를 말하는거임
    # 타일을 오픈
    global OPEN_COUNT
    if CHECKED[y_pos][x_pos]:  #이미 확인된 파일
        return                 # “이 함수, 여기서 끝. 이제 밖으로 나갈게.” CHECKED값이 True이면 
    
    CHECKED[y_pos][x_pos] = True  #함수의 무한반복 실행을 막기위함이다.

    for yoffset in range(-1,2):
        for xoffset in range(-1,2):
            xpos,ypos= (x_pos+xoffset, y_pos+yoffset)
            if 0 <= xpos < WIDTH  and 0 <= ypos < HEIGHT and field[ypos][xpos] == EMPTY:
                field[ypos][xpos] = OPENED
                OPEN_COUNT += 1

                count= num_of_bomb(field,xpos,ypos)
                if count ==0 and not(xpos==x_pos and ypos==y_pos):
                    open_tile(field,xpos,ypos)

def main():

    smallfont=  pygame.font.SysFont(None,36)
    largefont=  pygame.font.SysFont(None,72)
    message_clear= largefont.render("CLEARED!!", True,(0,255,255))
    message_over = largefont.render("Game Over!!", True,(0,255,255))

    message_rect = message_clear.get_rect()
    message_rect.center=(WIDTH*SIZE/2,HEIGHT*SIZE/2)

    field= [[EMPTY for xpos in range(WIDTH)] for ypos in range(HEIGHT)]
    game_over=False

    # 폭탄을 설치
    count = 0
    while count < NUM_OF_BOMBS:
        xpos,ypos= randint(0,WIDTH-1),randint(0,HEIGHT-1)
        if field[ypos][xpos] == EMPTY:
            field[ypos][xpos]= BOMB
            count +=1
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == MOUSEBUTTONDOWN and event.button==1:
                xpos,ypos= floor(event.pos[0]/SIZE),floor(event.pos[1]/SIZE)
                if field[ypos][xpos] == BOMB:
                    game_over= True
                else:
                    open_tile(field,xpos,ypos)

        # 그리기
        SURFACE.fill((10,0,0))
        for ypos in range(HEIGHT):
            for xpos in range(WIDTH):
                tile= field[ypos][xpos]
                rect= (xpos*SIZE,ypos*SIZE,SIZE,SIZE)

                if tile == EMPTY or tile==BOMB:
                    pygame.draw.rect(SURFACE,(192,192,192),rect)
                    
                    if game_over and tile ==BOMB:
                        pygame.draw.ellipse(SURFACE,(225,225,0),rect)
                elif tile==OPENED:
                    count = num_of_bomb(field,xpos,ypos)
                    if count > 0 :
                        num_image=smallfont.render(f"{count}",True,(255,255,0))
                        SURFACE.blit(num_image,(xpos*SIZE+10,ypos*SIZE+10))

               
        # 선그리기
        for i in range(0,WIDTH*SIZE,SIZE):
            pygame.draw.line(SURFACE,(96,96,96),(i,0),(i,HEIGHT*SIZE))

        for j in range(0,HEIGHT*SIZE,SIZE):
            pygame.draw.line(SURFACE,(96,96,96),(0,j),(WIDTH*SIZE,j))

        # 메세지 나타내기
        if OPEN_COUNT == WIDTH*HEIGHT  -  NUM_OF_BOMBS:
            SURFACE.blit(message_clear,message_rect.topleft)
        
        elif game_over:
            SURFACE.blit(message_over,message_rect.topleft)
            

        pygame.display.update()
        FPSCLOCK.tick(10)

if __name__ =='__main__':
    main()